# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Data manipulation example: Combining MIRI cubes into a mosaic

Michael Reefe

This example notebook provides a quick tutorial on how to use LOKI to combine multiple observations of a single target into a mosaicked cube, which can then be fit using all of the regular methods.

First things first, we need to import the LOKI code. Remember we need to activate our project first (refer to the installation section of the README). We can do so simply by:

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()

using Loki

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


Now we want to load in our data. For this example, we'll be using the data for the Sculptor Galaxy, which is located in the same folder as this notebook. 

We have three unique observations for this target at slightly different pointings, and we want to combine them into a single cube.  We first have to make Observation objects for each separate pointing.

In [2]:
# The redshift of the target object: NGC 253 (Sculptor)
z = 0.000807

# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
obs_1 = from_fits(["jw01701-c1002_t003_miri_ch3-medium_s3d.fits.gz"], z);
obs_2 = from_fits(["jw01701-o012_t004_miri_ch3-medium_s3d.fits.gz"], z);
obs_3 = from_fits(["jw01701-o013_t005_miri_ch3-medium_s3d.fits.gz"], z);

# This puts the observation in the rest frame, masks out bad spaxels, dereddens it, and logarithmically rebins it in wavelength
correct!(obs_1)
correct!(obs_2)
correct!(obs_3)

[ Info: Initializing DataCube struct from jw01701-c1002_t003_miri_ch3-medium_s3d.fits.gz
[ Info: Initializing DataCube struct from jw01701-o012_t004_miri_ch3-medium_s3d.fits.gz
[ Info: Initializing DataCube struct from jw01701-o013_t005_miri_ch3-medium_s3d.fits.gz

new_wave contains values outside the range in old_wave, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. 

[ Info: Using SFD98 dust map at (α=11.888509583333333°, δ=-25.287788888888883°): E(B-V)=0.018742157430445203

new_wave contains values outside the range in old_wave, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. 

[ Info: Using SFD98 dust map at (α=11.887772083333333°, δ=-25.28844444444445°): E(B-V)=0.018742245109099213

new_wave contains values outside the range in old_wave, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. 

[ Info: Using SFD98 dust map at (α=11.886998333333333°, δ=-25.289080555

Observation(Dict{Any, DataCube}(:B3 => DataCube{Vector{Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[13.33049244515671 μm, 13.332990429227687 μm, 13.335488881392724 μm, 13.33798780173955 μm, 13.340487190355889 μm, 13.342987047329485 μm, 13.345487372748115 μm, 13.347988166699555 μm, 13.350489429271594 μm, 13.35299116055206 μm  …  15.529812831116857 μm, 15.532722943045354 μm, 15.535633600296025 μm, 15.538544802971074 μm, 15.541456551172702 μm, 15.544368845003124 μm, 15.547281684564599 μm, 15.550195069959384 μm, 15.553109001289759 μm, 15.556023478658036 μm], Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}[NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ … NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹; NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN

Now we combine these two by calling the `combine_observations` function -- this will create a new `Observation` object within which each channel will be the combined/mosaicked result of the channels in the input cubes.  Note that you can combine as many cubes as you want this way.

The cubes are projected onto an optimal output WCS, which is calculated to cover the full extent of all of the input cubes and to be at the highest resolution of any of the input cubes.

If the input channels don't have exactly the same wavelength vector, they will also be resampled in wavelength to match each other.

You can combine multiple channels at a time with this method, but in this instance I only have channel 3 medium data.

In [3]:
obs_mosaic = combine_observations([obs_1, obs_2, obs_3]; order=1, name_out="Sculptor_mosaicked")

[ Info: Channel: B3
[ Info: Reprojecting all channels onto the optimal WCS
[ Info: Reprojecting channel 3 MEDIUM onto the optimal (74, 72) WCS grid...


Progress: 100%|█████████████████████████████████████████| Time: 0:01:33


[ Info: Reprojecting channel 3 MEDIUM onto the optimal (74, 72) WCS grid...


Progress: 100%|█████████████████████████████████████████| Time: 0:01:33


[ Info: Reprojecting channel 3 MEDIUM onto the optimal (74, 72) WCS grid...


Progress: 100%|█████████████████████████████████████████| Time: 0:01:37


Observation(Dict{Any, DataCube}(:B3 => DataCube{Vector{Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[13.33049244515671 μm, 13.332990429227687 μm, 13.335488881392724 μm, 13.33798780173955 μm, 13.340487190355889 μm, 13.342987047329485 μm, 13.345487372748115 μm, 13.347988166699555 μm, 13.350489429271594 μm, 13.35299116055206 μm  …  15.529812831116857 μm, 15.532722943045354 μm, 15.535633600296025 μm, 15.538544802971074 μm, 15.541456551172702 μm, 15.544368845003124 μm, 15.547281684564599 μm, 15.550195069959384 μm, 15.553109001289759 μm, 15.556023478658036 μm], Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}[NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ … NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹; NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN

Next, if we then wish to combine the data from all channels into a single cube, we can do this as well with the `combine_channels!` function, which is showcased in the other example notebooks.  We just need to specify now which channels we're combining, and give an output ID for the key of the combined channel.  I am not doing so in this case because we only have data in one channel, but just keep it in mind for future reference!

Our last pre-processing checklist items are to:
   1. Interpolate any NaNs in the data -- this is only done for spaxels that are less than 50% NaNs, the others are treated as a lost cause.
   2. Replace the pipeline-produced errors with statistically calculated errors (from the std.dev. of the residuals of a cubic spline spline)

In [4]:
channel = :B3

# One may optionally wish to smooth out the cube by replacing the spectrum in each spaxel with one extracted from an aperture the size of the PSF FWHM.
# This may help to reduce resampling artifacts from the 3D drizzle algorithm used by the STSci reduction pipeline.
# If you wish, uncomment the following line to enable the smoothing.
# (note: this function is part of the combine_channels! algorithm, so it's not shown explicitly in the other example notebooks, but combine_channels!
#  performs no smoothing unless the keyword argument "extract_from_ap" is provided and greater than 0)
# extract_from_aperture!(obs_mosaic, [channel], 1.)

# We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
interpolate_nans!(obs_mosaic.channels[channel])

# Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
# and replace the errors in the cube with these, since the provided errors are typically underestimated.
# You can skip this step if you wish to use the default errors.
calculate_statistical_errors!(obs_mosaic.channels[channel])

# Save the pre-processed data as a FITS file so it can be quickly reloaded later
save_fits(".", obs_mosaic, [channel]);

[ Info: Interpolating NaNs in cube with channel B3, band MEDIUM:
[ Info: Calculating statistical errors for each spaxel...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


[ Info: Writing FITS file from Observation object


Now, I'm not actually going to attempt to fit this cube, because it's not the point of this example, but I will show you what the mosaicked cube looks like with some of the visualization helper functions.

In [5]:
using Cosmology
cosmo = cosmology(;h=0.7, OmegaM=0.27, OmegaK=0., OmegaR=1e-5)

# The "plot_2d" function plots a 2D projection of the cube -- by default, this does an integration along the wavelength axis,
# but if the "slice" argument is specified, it can also show a single wavelength slice at the given coordinate.
# "intensity" and "err" booleans enable/disable plotting the intensity and the error
# "logᵢ" and "logₑ" if set to an integer  will make the colorbar logarithmic with that integer's base
# "colormap" changes the colormap (default: cubehelix) (you need to import python's matplotlib colormap package to use this)
# "name", if given, adds a plot title
# "z" is the redshift and must be provided to add a physical scale bar annotation to the plot, otherwise only an angular scale is added
# "cosmo" specifies the cosmology and it also must be provided along with "z" to add the physical scale bar annotation
# "aperture", if given, plots an aperture on top of the 2D map; use the make_aperture function to create an aperture
plot_2d(obs_mosaic.channels[channel], "Sculptor_mosaic_2d_integrated.pdf"; intensity=true, err=true, logᵢ=10, logₑ=nothing, 
        name=nothing, slice=nothing, z=obs_mosaic.z, cosmo=cosmo, aperture=nothing)
plot_2d(obs_mosaic.channels[channel], "Sculptor_mosaic_2d_slice.pdf"; intensity=true, err=true, logᵢ=nothing, logₑ=nothing, 
        name=nothing, slice=500, z=obs_mosaic.z, cosmo=cosmo, aperture=nothing)

# The "plot_1d" function plots a 1D projection of the cube -- by default, this does an integration along the 2 spatial axes,
# but if the "spaxel" argument is specified, it can also show a single spaxel's spectrum at the given coordinates.
# "intensity", "err", "name", and "logᵢ" work the same way as they do in "plot_2d", but there is no "logₑ" argument 
#       because logᵢ now affects both the intensity and the errors, which are placed on the same axes
# "linestyle" works like the matplotlib argument of the same name
plot_1d(obs_mosaic.channels[channel], "Sculptor_mosaic_1d_integrated.pdf"; intensity=true, err=true, logᵢ=10, spaxel=nothing, linestyle="-")
plot_1d(obs_mosaic.channels[channel], "Sculptor_mosaic_1d_spaxel.pdf"; intensity=true, err=true, logᵢ=false, spaxel=(37,37), linestyle="-")

Here are the output 2D plots, showing the intensity on the left and the error on the right.  Notice the odd shape of the field of view, which is caused by mosaicking these 3 cubes together.  If you open the fits file, you can scroll through the wavelength axis and see how the combined cube looks at all wavelength (i.e. with a tool like DS9).

![](./Sculptor_mosaic_2d_integrated.png)
![](./Sculptor_mosaic_2d_slice.png)

And now the 1D plots, first the integrated one, and then for a single spaxel.

![](./Sculptor_mosaic_1d_integrated.png)
![](./Sculptor_mosaic_1d_spaxel.png)

Note: I would not recommend trying to fit this cube yourself, because these data products appear to have strong residual fringing artifacts which have not been removed by the STSci pipeline (by default, the residual fringe removal step is disabled).